In [1]:
import pandas as pd
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, accuracy_score


data = pd.read_csv("./data/whole_data.csv")
data = data.iloc[:, 1:]
data.head()

,X0,X1,X2,X3,X4,X5,X6,X7,X8,X9,X10,X11,X12,X13,X14,X15,X16,Action,Time,Event
0,0.62,0.03,0.10,0.28,0.43,0.67,0.38,0.12,0.15,0.88,0.72,0.89,0.43,0.36,0.43,0.66,0.16,0,0.47,1
1,0.12,0.87,0.43,0.28,0.02,0.23,0.82,0.14,0.81,0.68,0.53,0.24,0.05,0.01,0.96,0.98,0.34,0,1.39,1
2,0.37,0.06,0.47,0.86,0.86,0.33,0.01,0.90,0.79,0.43,0.64,0.84,0.05,0.56,0.01,0.77,0.62,0,3.61,1
3,0.57,0.39,0.65,0.76,0.83,0.92,0.57,0.60,0.34,0.35,0.44,0.93,0.44,0.65,0.61,0.25,0.96,0,1.02,1
4,0.82,0.00,0.26,0.27,0.82,0.68,0.43,0.92,0.81,0.26,0.66,0.04,0.10,0.93,0.82,0.37,0.09,1,2.07,1


In [2]:
X = data.iloc[:, :17]
y = data.iloc[:, 17]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y)

In [3]:
dataset = lgb.Dataset(X_train, label=y_train)

params={}
params['learning_rate'] = 0.01
params['boosting_type'] = 'gbdt' #GradientBoostingDecisionTree
params['objective']     = 'binary' #Binary target feature
params['metric']        = 'binary_logloss' #metric for binary classification
params['max_depth']     = 30

clf = lgb.train(params, dataset, 500)
pred = clf.predict(X_test).round(0)

f1_score(y_test, pred)

0.8190476190476189

In [4]:
final_result = []

learning_rate_list = [0.01]
max_depth_list     = [5, 7, 9, 10, 12, 14, 15]

for lr in learning_rate_list:
    for md in max_depth_list:
        param = {}
        param['learning_rate'] = lr
        param['max_depth']     = md
        params['boosting_type'] = 'gbdt'
        params['objective']     = 'binary'
        params['metric']        = 'binary_logloss'
        
        clf = lgb.train(param, dataset, 10000)
        pred = clf.predict(X_test)
        pred = pred.round(0)
        
        score = f1_score(y_test, pred).round(4)
        final_result.append([lr, md, score])
        print(f"Result: {lr}-{md} / {score}")

final_result = pd.DataFrame(final_result, columns=['learning rate', 
                                                  'max depth', 
                                                  'f1 score'])
print(final_result)

Result: 0.01-5 / 0.8519
Result: 0.01-7 / 0.8302


KeyboardInterrupt: 

In [5]:
test_data = pd.read_csv("./data/testX.csv")
test_data = test_data.iloc[:, 1:]
test_data

,X0,X1,X2,X3,X4,X5,X6,X7,X8,X9,X10,X11,X12,X13,X14,X15,X16
0,0.00,0.10,0.40,0.03,0.46,0.31,0.50,0.15,0.55,0.06,0.68,0.51,0.82,0.57,0.28,0.77,0.74
1,0.52,0.75,0.56,0.78,0.28,0.05,0.52,0.45,0.26,0.05,0.43,0.21,0.88,0.97,0.18,0.05,0.97
2,0.45,0.78,0.47,0.46,0.80,0.51,0.23,0.28,0.88,0.58,0.47,0.41,0.87,0.79,0.73,0.55,0.08
3,0.58,0.29,0.68,0.97,0.20,0.43,0.60,0.68,0.37,0.63,0.83,0.67,0.83,0.18,0.49,0.75,0.54
4,0.38,0.02,0.58,0.51,0.58,0.15,0.45,0.27,0.56,0.09,0.98,0.50,0.39,0.88,0.81,0.18,0.17
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
281,0.31,0.55,0.91,0.37,0.24,0.57,0.33,0.81,0.30,0.09,0.22,0.54,0.88,0.49,0.10,0.53,0.47
282,0.39,0.52,0.19,0.36,0.50,0.08,0.08,0.50,0.42,0.52,0.86,0.13,0.80,0.85,0.98,0.27,0.95
283,0.98,0.87,0.45,0.99,0.94,0.81,0.31,0.00,0.94,0.73,0.70,0.91,0.45,0.34,0.86,0.84,0.58
284,0.43,0.69,0.04,0.59,0.44,0.57,0.66,0.13,0.17,0.40,0.91,0.25,0.25,0.22,0.42,0.79,0.61


In [10]:
params={}
params['learning_rate'] = 0.01
params['boosting_type'] = 'gbdt' #GradientBoostingDecisionTree
params['objective']     = 'binary' #Binary target feature
params['metric']        = 'binary_logloss' #metric for binary classification
params['max_depth']     = 10

clf = lgb.train(params, dataset, 10000)
# pred = clf.predict(test_data).round(0)
# pred

array([0., 0., 0., 0., 0., 1., 1., 0., 1., 1., 1., 0., 1., 1., 0., 0., 1.,
       0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 1.,
       1., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 1., 1., 0., 0., 0., 1.,
       1., 1., 0., 1., 1., 1., 0., 0., 0., 0., 1., 0., 1., 0., 0., 1., 1.,
       0., 0., 0., 0., 0., 0., 1., 0., 0., 1., 0., 1., 1., 0., 0., 0., 1.,
       0., 1., 0., 1., 0., 0., 0., 1., 1., 0., 1., 1., 0., 1., 1., 1., 1.,
       0., 0., 1., 0., 1., 1., 1., 0., 0., 0., 0., 0., 0., 1., 1., 1., 0.,
       0., 1., 0., 0., 1., 0., 0., 1., 0., 0., 0., 0., 0., 1., 0., 1., 0.,
       0., 0., 0., 0., 1., 0., 1., 1., 0., 1., 1., 0., 1., 1., 0., 0., 0.,
       0., 1., 1., 1., 1., 0., 0., 1., 1., 1., 0., 0., 1., 0., 1., 1., 1.,
       0., 1., 0., 0., 0., 0., 1., 0., 1., 0., 0., 1., 0., 1., 0., 1., 0.,
       0., 1., 1., 1., 1., 0., 0., 1., 0., 1., 1., 0., 1., 1., 0., 0., 0.,
       0., 1., 0., 1., 0., 0., 1., 0., 0., 1., 1., 1., 0., 0., 1., 0., 0.,
       0., 1., 0., 0., 0.

In [19]:
pred = clf.predict(test_data).round(2)
pred = np.array(list(map(lambda x: 1 if x > 0 else 0, pred)))
pred

array([1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1,
       1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0,
       0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1,
       1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0,
       1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0,
       1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1,
       1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0,
       1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0,
       1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1,
       1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0,
       1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1,
       0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1,
       1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0])

In [20]:
result = pd.DataFrame(zip(list(range(287)), map(int, pred.tolist())), columns=['Title', 'action'])
result.to_csv('./result/lgb_result_change_thres.csv', index=False)
result

,Title,action
0,0,1
1,1,1
2,2,0
3,3,0
4,4,0
...,...,...
281,281,1
282,282,0
283,283,0
284,284,0


In [21]:
import numpy as np

np.sum(result.action == 1) / len(result.action) * 100

58.04195804195804

In [43]:
params = {'learning_rate': 0.01, 
          'max_depth': 16, 
          'boosting': 'gbdt', 
          'objective': 'regression', 
          'metric': 'mse', 
          'is_training_metric': True, 
          'num_leaves': 144, 
          'feature_fraction': 0.9, 
          'bagging_fraction': 0.7, 
          'bagging_freq': 5, 
          'seed':2018}

regressor = lgb.train(params, dataset, 10000)
pred = regressor.predict(X_test)

from sklearn.metrics import mean_squared_error
import numpy as np
np.sqrt(mean_squared_error(y_test, pred))

0.3201440277717884